# Box Canyon Park 

https://www.google.com/maps/place/Box+Canyon+Park/@33.8857385,-117.7309815,910m/data=!3m2!1e3!4b1!4m6!3m5!1s0x80dcce2292288e59:0xc1684d99397ec327!8m2!3d33.8857385!4d-117.7284066!16s%2Fg%2F1tm8kwym?entry=ttu

## 1. Organize data

Create a folder under `semantic_SfM/data` and organize your data following the structures below. 

Using SfM products from Agisoft:
```
semantic_SfM/data
    ├── box_canyon_park
        ├── DJI_photos
        │       ├── DJI_0000.JPG
        │       ├── DJI_0001.JPG
        │       ├── ...
        │       └── DJI_0100.JPG
        ├── SfM_products
        │       ├── agisoft_cameras.xml
        │       ├── model.jpg
        │       ├── model.obj
        │       ├── model.mtl
        │       └── agisoft_model.las   
        ├── segmentations
        └── associations

```

## 2. Create 2D Segmentation using SAM

In [ ]:
from ssfm.image_segmentation import ImageSegmentation
import os

In [ ]:
sam_params = {}
sam_params['model_name'] = 'sam'
sam_params['model_path'] = '../semantic_SfM/sam/sam_vit_h_4b8939.pth'
sam_params['model_type'] = 'vit_h'
sam_params['device'] = 'cuda:1'
sam_params['points_per_side'] = 64
sam_params['pred_iou_thresh'] = 0.96
sam_params['stability_score_thresh'] = 0.92
sam_params['crop_n_layers'] = 2

In [ ]:
image_segmentor = ImageSegmentation(sam_params)   
image_segmentor.set_distortion_correction('../data/box_canyon_park/SfM_products/agisoft_cameras.xml')
image_folder_path = '../data/box_canyon_park/DJI_photos'
segmentation_folder_path = '../data/box_canyon_park/segmentations'
image_paths = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
image_segmentor.batch_predict(image_paths, segmentation_folder_path, maximum_size=1000, save_overlap=True)

## 3. Create projection associations

In [ ]:
from ssfm.probabilistic_projection import *
import time

In [ ]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=0.2)
#pointcloud_projector = PointcloudProjection()

In [ ]:
pointcloud_projector.read_camera_parameters('../data/box_canyon_park/SfM_products/agisoft_cameras.xml')
pointcloud_projector.read_mesh('../data/box_canyon_park/SfM_products/model.obj')
pointcloud_projector.read_pointcloud('../data/box_canyon_park/SfM_products/agisoft_model.las')

In [ ]:
# batch project
image_folder_path = '../data/box_canyon_park/DJI_photos'
save_folder_path = '../data/box_canyon_park/associations'

image_list = [f for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
pointcloud_projector.parallel_batch_project_joblib(image_list, save_folder_path, num_workers=8)

In [ ]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [ ]:
smc_solver = KeyimageAssociationsBuilder('../data/box_canyon_park/associations', '../data/box_canyon_park/segmentations')
smc_solver.build_associations()
smc_solver.find_min_cover()
smc_solver.refine(0.5)

## 4. Estimate memory usage

In [ ]:
from ssfm.memory_calculator import memory_calculator

In [ ]:
# pointcloud file
las_file = "../data/box_canyon_park/SfM_products/agisoft_model.las"
# image file sample; this needs to be an original image even if patch images are used
image_file = "../data/box_canyon_park/DJI_photos/DJI_0350.JPG"
# number of images
num_images = 129
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(las_file, image_file, num_images, num_segmentation_ids)

## 5. Run object registration

In [1]:
from ssfm.object_registration import *
from ssfm.post_processing import PostProcessing
import time

In [ ]:
# Set paths
pointcloud_path = '../data/box_canyon_park/SfM_products/agisoft_model.las'
segmentation_folder_path = '../data/box_canyon_park/segmentations'
image_folder_path = '../data/box_canyon_park/DJI_photos_split'
association_folder_path = '../data/box_canyon_park/associations'

keyimage_associations_file_name= 'refined_associations_keyimage.npy'
keyimage_yaml_name= 'refined_keyimages.yaml'

# Create object registration
t1 = time.time()
obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path, keyimage_associations_file_name, keyimage_yaml_name)
t2 = time.time()
print('Time elapsed for creating object registration: {}'.format(t2-t1))

# Run object registration
obr.object_registration(iou_threshold=0.5, save_semantics=True)
#obr.object_registration(iou_threshold=0.5)

# save semantic point cloud
obr.save_semantic_pointcloud('../data/box_canyon_park/associations/semantic_model.las')

In [ ]:
# shuffle semantic ids
semantic_pc_file_path = '../../data/box_canyon_park/semantic_model.las'
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = '../../data/box_canyon_park/semantic_model_shuffled.las'
post_processing.save_semantic_pointcloud(save_las_path)

In [ ]:

pointcloud_path = '../../data/box_canyon_park/SfM_products/agisoft_model.las'
segmentation_folder_path = '../../data/box_canyon_park/segmentations'
image_folder_path = '../../data/box_canyon_park/DJI_photos'
association_folder_path = '../../data/box_canyon_park/associations'

obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path)

add_semantics_to_pointcloud_flag = True
if add_semantics_to_pointcloud_flag:
    image_id = 257
    semantics_folder_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
    save_las_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}_depth_filter.las'.format(image_id))
    add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path) 

In [6]:
# shuffle semantic ids
semantic_pc_file_path = '../data/box_canyon_park/associations/semantics/semantics_257_depth_filter.las'
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = '../data/box_canyon_park/semantic_model_shuffled.las'
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  1880
